# Sistemas Inteligentes

## Curso académico 2024-2025

### Laboratorio 2: Búsqueda Metaheurística

#### Instructores

* Juan Carlos Alfaro Jiménez: JuanCarlos.Alfaro@uclm.es
* María Julia Flores Gallego: Julia.Flores@uclm.es
* Ismael García Varea: Ismael.Garcia@uclm.es
* Adrián Rodríguez López: Adrian.Rodriguez18@alu.uclm.es

## Estaciones de Servicio y Energía: Encontrando la Configuración Óptima

## 1. Introducción

¡Noticias emocionantes! El **Ministerio de Transporte y Movilidad Sostenible** ha quedado muy impresionado con las soluciones desarrolladas en nuestro primer trabajo, la Práctica 1. Están particularmente interesados en implementar estos algoritmos en la planificación de rutas de vehículos autónomos, con A* como el método MÁS efectivo para identificar el camino óptimo de manera eficiente. Para avanzar en este proyecto, el Ministerio tiene como objetivo establecer estratégicamente estaciones de servicio en áreas urbanas para apoyar su flota de vehículos autónomos. Estas estaciones funcionarán como centros de flota y proporcionarán servicios esenciales para los vehículos.

Para lograr esto, han solicitado **nuestra experiencia técnica para determinar la distribución óptima de estas estaciones** en los mapas de la ciudad. Sin embargo, no todas las intersecciones son elegibles como ubicación de una estación; el Ministerio ha preseleccionado intersecciones candidatas basándose en criterios específicos establecidos por sus equipos técnicos y administrativos. Su principal enfoque es la sostenibilidad y el acceso equitativo, con el objetivo de garantizar que todos los ciudadanos estén razonablemente cerca de una estación de servicio. Entre estos puntos seleccionados, solo se elegirá un número específico. Para facilitar nuestra tarea de determinar cuáles deben ser, han proporcionado datos sobre la cobertura poblacional de cada intersección candidata, lo que nos permite tener en cuenta tanto el acceso como la cobertura en nuestra estrategia de distribución.

El objetivo principal es garantizar un acceso eficiente a la máxima población posible, manteniendo una distribución equilibrada en toda la ciudad, una consideración vital para un sistema de transporte completamente autónomo.

### 1.1 Objetivos del Laboratorio

En esta práctica, aplicaremos técnicas de búsqueda metaheurística para resolver problemas de optimización combinatoria.

El primer objetivo es comprender la tarea y formularla desde la perspectiva de la búsqueda metaheurística. Implementaremos al menos dos algoritmos:

* **Búsqueda Aleatoria**, como un punto de partida básico que generará múltiples soluciones, evaluará cada una y devolverá la mejor.

* **Algoritmo Genético**, que permitirá configurar varios parámetros, como el tamaño de la población, la tasa de mutación y el número de generaciones, entre otros.


Además, para la evaluación no-continua se tendrá que implementar la Ascensión de Cplinas o Hill Cilimbing (obligatoriamente), y opcionalmente implementar el algoritmo **Iterated Local Search** (ILS), ambos explicados en el Tema 7.

A continuación, analizaremos y compararemos el rendimiento de estos algoritmos ejecutándolos en instancias de problemas de diferente complejidad.

Esperamos que esta práctica te ayude a profundizar en tu comprensión de las técnicas metaheurísticas y te anime a considerar cómo se pueden aplicar en problemas reales de optimización combinatoria.

**¡Buena suerte!**

## 2. Descripción del Problema

### 2.1 Problemas de Entrada

Cada escenario se proporcionará en un archivo en formato `json` que contiene la siguiente información, con el formato de un diccionario cuyas claves son:

* `address`: La dirección utilizada.
* `distance`: Radio máximo utilizado para definir intersecciones y segmentos alrededor de la dirección.
* `intersections`: Una lista de diccionarios con información sobre las intersecciones.
* `segments`: Una lista de diccionarios con información sobre los segmentos, que representan las calles entre dos intersecciones.
* `candidates`: Una lista de pares (identificador, población) que contiene las intersecciones candidatas. Notad que los identificadores en esta lista deben estar incluidos en la lista de intersecciones.
* `number_stations`: El número de estaciones de servicio que se deben ubicar, que no debe superar el número de candidatos.

Cada diccionario en `intersections` incluye tres claves:

* `identifier`: Identificador de la intersección
* `longitude`: Longitud de la intersección
* `latitude`: Latitud de la intersección

Cada diccionario en `segments` incluye cuatro claves:

* `origin`: Intersección de origen
* `destination`: Intersección de destino
* `distance`: Distancia entre las dos intersecciones
* `speed`: Velocidad máxima permitida entre las dos intersecciones

**IMPORTANTE**: `initial` y `final` ya no están incluidos en el archivo JSON, ya que no son necesarios. Durante la evaluación de una posible configuración, estos puntos iniciales y finales cambiarán varias veces. Esto puede requerir algunos ajustes en el código de la Práctica 1 para ejecutar A*. Estos cambios deben estar claramente indicados (tu código debe coincidir con el de la Práctica 1, excepto por estos cambios) y discutidos en la memoria de prácticas.

### 2.2. Ejemplo ilustrativo

Un posible ejemplo de este problema podría ser el que se muestra en la siguiente imagen, que representa una parte de la ciudad de Albacete:

![title](sample-problems-lab2/toy/example.png)

En este caso, el número de estaciones de servicio de vehículos que se deben ubicar es 4, entre las 15 intersecciones candidatas representadas con puntos azules (etiquetadas con la población cubierta). Una posible solución se representa con puntos verdes.

---

##### Nota:

* El archivo que contiene la imagen debe guardarse en la ruta indicada en el código de esta celda.

---

### 2.3 Definición formal del problema

Necesitamos elegir $s$ estaciones de entre $c$ intersecciones candidatas o elegibles, con $s<c$. Por lo tanto, nuestro objetivo es decidir en cuál de estas $c$ intersecciones candidatas debemos ubicar las $s$ estaciones de servicio de vehículos, de manera que se minimice el tiempo promedio de viaje que cada habitante tarda desde su hogar hasta la estación más cercana. Si denotamos por $S$ al vector de tamaño $s$ que contiene las intersecciones en las que se ubican las estaciones de vehículos y por $C$ al vector de intersecciones candidatas que contiene el par (id, pop) para cada intersección candidata, entonces formalmente, queremos resolver el siguiente problema de optimización:

$$
S^* = \arg\min_{S} \frac{1}{\sum_{i=0}^{c-1} C[i].pop} \cdot \min_{j=0,\dots,s-1} \left\{\sum_{i=0}^{c-1} \; C[i].pop \cdot time(C[i].id,S[j])\right\}
$$

donde:
- $C[i].pop$ representa la población (número de habitantes) cubierta por la intersección candidata $i$.
- $C[i].id$ es el identificador de la intersección candidata $i$.
- $time(A,B)$ representa el menor tiempo real para viajar desde la intersección $A$ hasta la intersección $B$.

Las siguientes consideraciones deben tenerse en cuenta respecto a la expresión anterior:
- Estamos tratando con un problema de minimización.
- La cardinalidad del espacio de búsqueda es:

$$
\binom{c}{s} = \frac{c!}{(c-s)!s!}
$$

por ejemplo, si tenemos 20 intersecciones elegibles y 4 estaciones de vehículos, el número de soluciones posibles es 210, no demasiadas; pero si tenemos 100 candidatos y 10 estaciones, entonces el número de soluciones posibles es $1.7\times10^{13}$ ($5.3\times 10^{20}$ con 20 estaciones).

## 3. Desarrollo de la práctica

Antes de implementar los algoritmos, primero debes considerar definir los elementos básicos en este tipo de problemas, a saber:

- Una representación conveniente para las soluciones (configuraciones, cromosomas, individuos, ...) del problema que se utilizarán en los algoritmos de optimización combinatoria. Piensa detenidamente en las distintas opciones y tendrás que discutirlas en el informe de la tarea.

- Implementar un mecanismo de evaluación para gestionar las evaluaciones realizadas por los algoritmos de optimización combinatoria. A continuación, se detallará cómo debe realizarse la evaluación.

- Notas importantes:
    - En el caso de que A* no devuelva ninguna solución (coste = inf), reemplazad este valor por un número muy alto en comparación con el tiempo máximo en nuestro problema. Reflexiona sobre la necesidad de esto y discútelo en el informe.
    - Podéis aprovechar el mecanismo de evaluación para guardar algunos cálculos, recopilar estadísticas e imprimir los resultados.
    - Tened en cuenta que esta tarea requiere que ya hayas resuelto la Práctica 1, y necesitarás reutilizar el código implementado para resolver esta práctica.
   
Tendréis que resolver muchos problemas similares a los de la Práctica 1. Los mapas serán los mismos, pero los problemas necesitan incorporar nueva información, que es la lista de intersecciones candidatas y, para cada una de ellas, la población que cubren. El número de estaciones que se deben ubicar también se indica en el problema como `number_stations`.


### 3.1 Evaluación de una solución

Dada una instancia específica del problema a resolver, y asumiendo que $C$ denota su lista de intersecciones candidatas, el valor de cada posible solución $S$ debe calcularse como:

$$value(S) = \frac{1}{\sum_{i=0}^{c-1} C[i].pop} \cdot \min_{j=0,\dots,s-1} \left\{\sum_{i=0}^{c-1} \; C[i].pop \cdot time(C[i].id,S[j])\right\}$$

de acuerdo con la fórmula presentada en la sección 2.2.

## 4. Plan de trabajo

### 4.1. Tareas

* Transferid y adaptad vuestro código de la Práctica 1 para resolver búsquedas con A* que necesitaréis aquí:
    * Reutilizad la mayor parte del código necesario de vuestra Práctica 1.
    * Describid qué se ha modificado, por qué y cómo.

* Procesad los nuevos archivos JSON y guardad el problema de acuerdo con lo siguiente:
    * Además de las clases de búsqueda (Problem_2, State, Action, ...), deberéis trabajar con las intersecciones candidatas.
    * Construid un mecanismo capaz de almacenar y recuperar las intersecciones candidatas y la población asociada a cada una.

* Representación de una posible configuración:
    * Entre las vistas en el Tema 6, encontrad la representación más adecuada para este problema y adaptadla, teniendo en cuenta que cada problema tiene valores distintos para el número total de intersecciones, intersecciones elegibles y el número solicitado de estaciones.
    * Conectadla a un método adecuado para evaluar cada una considerando las indicaciones mencionadas anteriormente en el punto 3.2.

* Implementación de algoritmos:
    * Implementad, al menos, los dos algoritmos requeridos (Búsqueda Aleatoria y un Algoritmo Genético — GA). Tened en cuenta que para la evaluación no continua, también deberéis agregar Hill Climbing y, opcionalmente, ILS.
    * En el GA, aseguraos de haber implementado la generación de una población junto con los elementos principales dentro del bucle principal: selección, cruce, mutación y combinación de generaciones.

* Experimentación y análisis:
    * Los parámetros que se puedan ajustar deben ser explorados adecuadamente, también en relación con los problemas dados (dimensionalidad, complejidad, etc.).
    * También deberéis estudiar el rendimiento resultante en términos de desempeño, convergencia, número de generaciones, etc.
    * Comparad la Búsqueda Aleatoria y el Algoritmo Genético, asegurándoos de obtener resultados consistentes.

* Informe:
    * Redactad un informe detallando el proceso seguido, las estrategias implementadas y los resultados obtenidos, junto con gráficos y comparaciones visuales.


### 4.2. Evaluación de la práctica

En la modalidad de **evaluación continua**, la evaluación de la práctica se realizará a través de un examen individual en el que se tendrá en cuenta lo siguiente:

* Definición e implementación correcta de la representación de la configuración y de la función de evaluación: 25%
* Implementación correcta del algoritmo genético: 50%, que cubre
    * El bucle general para las generaciones es correcto: 10%
    * Los distintos operadores están correctamente diseñados y codificados: 40%
* Eficiencia y optimización: 15%
* Experimentación realizada y análisis de resultados: 10%

Es necesario que la Práctica 1 esté correctamente integrada y que la Búsqueda Aleatoria funcione de manera consistente para que todos los estudiantes puedan utilizarla como un punto de partida base adecuado.

Todo esto se ponderará según el nivel de conocimiento que el estudiante demuestre sobre la práctica en caso de que el examen sea una entrevista personal.

En la modalidad de **evaluación no continua**, la evaluación se modificará como se indica a continuación:

* Definición e implementación correcta de la representación de la configuración y de la función de evaluación: 15%
* Implementación correcta del algoritmo genético: 40%, que cubre
    * El bucle general para las generaciones es correcto: 7%
    * Los distintos operadores están correctamente diseñados y codificados: 33%
* Implementación correcta del algoritmo de Hill Climbing (obligatorio): 15%
* Implementación correcta del algoritmo ILS (opcional): 5%
* Eficiencia y optimización: 15%
* Experimentación realizada y análisis de resultados: 10%


### 4.3. Fechas importantes

* Fecha límite para entregar el código: **13 de diciembre de 2024**.
* Fecha límite para la entrega del informe: **Final del semestre**.


In [202]:
import json
from queue import PriorityQueue
import random
from geopy import distance
from abc import ABC, abstractmethod
from math import sqrt


def load_data(fileName):
    if not fileName.endswith(".json"):
        print("El archivo no es válido.")
        return
    with open(fileName, 'r') as file:
        data = json.load(file)
    return data

class Estado:

    def __init__(self, id, long, lat):
        self.identificador = id
        self.longitud = long
        self.latitud = lat

class Accion:

    def __init__(self, orig, dest, dist, vel):
        self.origen = orig
        self.destino = dest
        self.coste = dist / toMetersPerSecond(vel)
    
    def __lt__(self, otro):
        return self.destino < otro.destino

class Nodo:

    def __init__(self, est, pad, acc):
        self.estado = est
        self.padre = pad
        self.accion = acc
        if self.padre is not None and self.accion is not None:
            self.profundidad = self.padre.profundidad + 1
            self.coste = self.padre.coste + self.accion.coste
        else:
            self.profundidad = 0
            self.coste = 0

    def __lt__(self, otro):
        return self.estado.identificador < otro.estado.identificador

class Problema:

    def __init__(self, problemName):
        self.problemName = problemName
        self.data = load_data(self.problemName)
        if not self.data:
            print("[ERROR] No se ha encontrado la información del problema.")
            return
        # Encontramos el estado final y el estado inicial en el diccionario de estados
        self.calcular_acciones()
        self.calcular_estados()
        self.candidatos = self.data['candidates']
        self.numero_estaciones = self.data['number_stations']
    
    # Diccionario de acciones para calcular las conexiones entre intersecciones
    def calcular_acciones(self):
        self.acciones = {}
        self.velocidad_maxima = 0
        for element in self.data['segments']:
            if element['origin'] not in self.acciones:
                self.acciones[element['origin']] = []
            self.acciones[element['origin']].append(Accion(element['origin'], element['destination'], element['distance'], element['speed']))
            if element['speed'] > self.velocidad_maxima:
                self.velocidad_maxima = element['speed']


    # Diccionario de estados, id del estado, estado en el otro lado
    def calcular_estados(self):
        self.estados = {}
        for element in self.data['intersections']:
            self.estados[element['identifier']] = Estado(element['identifier'], element['longitude'], element['latitude'])
    
class Heuristica():
    def __init__(self, valor):
        self.heuristica = valor
    
    def funcion_heuristica(self, latitudFinal, latitudInicial, longitudFinal, longitudInicial):
        return sqrt((abs(latitudFinal - latitudInicial)**2 + abs(longitudFinal - longitudInicial)**2)) / toMetersPerSecond(self.heuristica)

class Busqueda(ABC):

    
    def __init__(self, problema, frontera):
        self.problema = problema
        self.frontera = frontera

    def start(self):
        self.solucion = self.algoritmo()
        
    def algoritmo(self):
        self.cerrados = set()
        nodoInicial = Nodo(self.problema.estado_inicial, None, None)
        self.insertar(nodoInicial)
        while(True):
            if self.es_vacia():
                return 3600*5
            self.nodoActual = self.sacar_siguiente()
            if self.es_final():
                return self.nodoActual.coste
            if self.nodoActual.estado not in self.cerrados:
                self.cerrados.add(self.nodoActual.estado)
                self.abrir_nodo()

    @abstractmethod
    def insertar(self, nodo):
        pass
    
    def es_vacia(self):
        return self.frontera.empty()
    
    @abstractmethod
    def sacar_siguiente(self):
        pass    

    def es_final(self):
        return self.nodoActual.estado == self.problema.estado_final

    def abrir_nodo(self):
        if self.nodoActual.estado.identificador in self.problema.acciones:
            acciones = self.problema.acciones[self.nodoActual.estado.identificador]
        else:
            return
        for element in acciones:
            accion = element
            nodoFrontera = Nodo(self.problema.estados[accion.destino], self.nodoActual, accion)
            self.insertar(nodoFrontera)

class AE(Busqueda):

    def __init__(self, problema, heuristica):
        frontera = PriorityQueue()
        super().__init__(problema, frontera)
        self.heuristica = heuristica

    def insertar(self, nodo):
        prioridad = self.heuristica.funcion_heuristica(self.problema.estado_final.latitud, nodo.estado.latitud, self.problema.estado_final.longitud, nodo.estado.longitud) + nodo.coste
        self.frontera.put((prioridad, nodo))

    def sacar_siguiente(self):
        return self.frontera.get()[1]


class Individuo():

    cache = {}
    hits = 0
    misses = 0

    def __init__(self, seleccionados=None, candidatos=[], tamano=0):
        if seleccionados is None:
            self.seleccionados = tuple()
            self.generar(candidatos, tamano)
        else:
            self.seleccionados = tuple(seleccionados)
        self.candidatos = candidatos
        self.tamano = tamano

    def evaluar(self, problema):
        sumPop = 0
        aux = 0
        estrella = AE(problema, Heuristica(problema.velocidad_maxima))

        for candidato in problema.candidatos:
            minimum = 3600*5
            for solucion in self.seleccionados:
                tupla = (candidato[0], solucion)
                if tupla in Individuo.cache:
                    val = Individuo.cache[tupla]
                    Individuo.hits += 1
                else:
                    estrella.problema.estado_inicial = problema.estados[candidato[0]]
                    estrella.problema.estado_final = problema.estados[solucion]
                    estrella.start()
                    val = estrella.solucion
                    Individuo.cache[tupla] = val
                    Individuo.misses += 1
                minimum = min(val, minimum)
            sumPop += candidato[1]
            aux += candidato[1]*minimum
        self.fitness = (1/sumPop) * aux

    def cruzar(self, otro, probabilidad=1.0):

        print(f"Padre1: {self.seleccionados}")
        print(f"Padre2: {otro.seleccionados}")

        if random.random() < probabilidad:
            corte = random.randint(1, len(self.seleccionados) - 1)
            piscina_genetica = self.seleccionados + otro.seleccionados
            hijo1 = list(self.seleccionados[corte:] + otro.seleccionados[:corte])
            hijo2 = list(otro.seleccionados[corte:] + self.seleccionados[:corte])
            if len(hijo1) < len(self.seleccionados):
                faltantes = len(self.seleccionados) - len(hijo1)
                hijo1.append(x for x in random.sample(piscina_genetica - hijo1, faltantes))
            if len(hijo2) < len(self.seleccionados):
                faltantes = len(self.seleccionados) - len(hijo2)
                hijo1.append(x for x in random.sample(piscina_genetica - hijo2, faltantes))
        else:
            hijo1 = self.seleccionados
            hijo2 = otro.seleccionados

        print(f"hijo1: {hijo1}")
        print(f"hijo2: {hijo2}")

        return (Individuo(hijo1, candidatos=self.candidatos), Individuo(hijo2, candidatos=self.candidatos))
                    
    def mutar(self, probabilidad=1.0):
        # Aplica una mutación a partir de una probabildad dada.
        aux = list(self.seleccionados)
        for _ in range(len(self.seleccionados)):
            if random.random() < probabilidad:
                nuevo = random.choice(self.candidatos)[0]
                while nuevo in aux:
                    nuevo = random.choice(self.candidatos)[0]
                aux.remove(random.choice(aux))
                aux.append(nuevo)
        self.seleccionados = tuple(aux)
    
    def generar(self, candidatos, tamano):
        list = random.sample(candidatos, tamano)
        self.seleccionados = tuple(x[0] for x in list)


class AlgoritmoAleatorio():

    cache = {}

    def __init__(self, problema, tamano_poblacion=100):
        self.problema = problema
        self.tamano_poblacion = tamano_poblacion

    def algoritmo(self):
        mejor_individuo = None

        for i in range(self.tamano_poblacion):
            temp = Individuo(candidatos=self.problema.candidatos, tamano=self.problema.numero_estaciones)

            acceso = temp.seleccionados

            if acceso in AlgoritmoAleatorio.cache:
                fitness = AlgoritmoAleatorio.cache[acceso]
            else:
                temp.evaluar(self.problema)
                fitness = temp.fitness
                AlgoritmoAleatorio.cache[acceso] = fitness
            
            if mejor_individuo is None:
                mejor_individuo = temp
            
            if mejor_individuo.fitness > fitness:
                mejor_individuo = temp

        return mejor_individuo


class AlgoritmoGenetico:

    cache = {}

    def __init__(self, problema, tamano_poblacion=100, generaciones=50, probabilidad_cruce=0.7777, probabilidad_mutacion=0.7777):
        self.problema = problema
        self.tamano_poblacion = tamano_poblacion
        self.generaciones = generaciones
        self.probabilidad_cruce = probabilidad_cruce
        self.probabilidad_mutacion = probabilidad_mutacion

    def seleccionar_rango(self, poblacion):

        N = len(poblacion)
        seleccionados = []

        prob = [2 * (N - i + 1) / (N**2 + N) for i in range(0, N)]

        seleccionados = random.choices(population=poblacion, weights=prob, k=random.randint(5, N))

        return seleccionados

    def seleccionar_torneo(self, poblacion, k=2):
        seleccionados = []
        for _ in range(len(poblacion)):
            torneo = random.sample(poblacion, k)
            ganador = max(torneo, key=lambda i : i.fitness)
            seleccionados.append(ganador)
        return seleccionados



    def algoritmo(self):

        mejor_individuo = None


        # inicializar P(t)
        poblacion = [Individuo(candidatos=self.problema.candidatos, tamano=self.problema.numero_estaciones) for _ in range(self.tamano_poblacion) ]

        # evaluar P(t)
        for element in poblacion:
            acceso = element.seleccionados
            if acceso in AlgoritmoGenetico.cache:
                element.fitness = AlgoritmoGenetico.cache[acceso]
            else:
                element.evaluar(self.problema)
                AlgoritmoGenetico.cache[acceso] = element.fitness
        
        poblacion.sort(key=lambda i : i.fitness, reverse=True)
        
        mejor_individuo = poblacion[0]

        for gen in range(self.generaciones):
            # poblacion_auxiliar = self.seleccionar_torneo(poblacion, random.randint(1, self.tamano_poblacion))
            poblacion_auxiliar = self.seleccionar_rango(poblacion)

            nueva_poblacion = []

            N = self.tamano_poblacion
            for i in range(0, len(poblacion_auxiliar), 2):
                if i + 1 < len(poblacion_auxiliar):
                    


                    hijos = poblacion_auxiliar[i].cruzar(poblacion_auxiliar[i+1], probabilidad=self.probabilidad_cruce)
                    hijo1 = hijos[0]
                    hijo2 = hijos[1]

                    hijo1.mutar(self.probabilidad_mutacion)
                    hijo2.mutar(self.probabilidad_mutacion)

                    acceso1 = hijo1.seleccionados
                    if acceso1 in AlgoritmoGenetico.cache:
                        hijo1.fitness = AlgoritmoGenetico.cache[acceso1]
                    else:
                        hijo1.evaluar(self.problema)
                        AlgoritmoGenetico.cache[acceso1] = hijo1.fitness

                    acceso2 = hijo2.seleccionados
                    if acceso2 in AlgoritmoGenetico.cache:
                        hijo2.fitness = AlgoritmoGenetico.cache[acceso2]
                    else:
                        hijo2.evaluar(self.problema)
                        AlgoritmoGenetico.cache[acceso2] = hijo2.fitness

                    nueva_poblacion.extend([hijo1, hijo2])

                else:
                    nueva_poblacion.append(poblacion_auxiliar[i])

            combinada = poblacion + nueva_poblacion


            poblacion = sorted(combinada, key= lambda i : i.fitness, reverse=True)[:self.tamano_poblacion]

            if mejor_individuo is None:
                mejor_individuo = poblacion[0]
            elif mejor_individuo.fitness < poblacion[0].fitness:
                mejor_individuo = poblacion[0]

            print(f"Generación: {gen}")
            print(f"Mejor Individuo: {mejor_individuo.seleccionados}")
            print(f"Hits en caché Individuo: {Individuo.hits}")
            print(f"Fallos en caché Individuo: {Individuo.misses}")



        return mejor_individuo
                
            




# def imprimirResultado(busqueda):
#     print(f"Nodos generados: {busqueda.generados}")
#     print(f"Nodos expandidos: {busqueda.expandidos}")
#     print(f"Nodos explorados: {busqueda.explorados}")
#     tiempo = datetime.timedelta(seconds=busqueda.tiempoEjecucion)
#     print(f"Duración de la ejecución: {tiempo}")
#     coste = datetime.timedelta(seconds=busqueda.coste)
#     print(f"Coste final: {coste}")
#     reconstruirCamino(busqueda.solucion)

# def reconstruirCamino(nodo):
#     if nodo is None:
#         return
#     ids = [nodo.estado.identificador]
#     while nodo.padre is not None:
#         nodo = nodo.padre
#         ids.append(nodo.estado.identificador)
#     ids.reverse()
#     print(f"Longitud de la solucion: {len(ids)}")
#     print(f"Camino recorrido: {ids}")




def toMetersPerSecond(kilometersPerHour):
    return (kilometersPerHour * 1000) / 3600

def main():
    prob = Problema("sample-problems-lab2/medium/calle_f_albacete_2000_0_candidates_25_ns_4.json")
    # AA = AlgoritmoAleatorio(prob)
    # print(AA.algoritmo().seleccionados)
    # print("-------------------------------------------------------------------")
    AG = AlgoritmoGenetico(prob, 100)
    print(f"Solución: {AG.algoritmo().seleccionados}")

if __name__ == "__main__":
    main()



Padre1: (7350762742, 1557019966, 5582955061, 2161030013)
Padre2: (4221333605, 1557019715, 4221333375, 1563381063)
hijo1: [2161030013, 4221333605, 1557019715, 4221333375]
hijo2: [1563381063, 7350762742, 1557019966, 5582955061]
Padre1: (1773309963, 2161030013, 5582955061, 7350762742)
Padre2: (2163186255, 5582955075, 1788442041, 2161030013)
hijo1: [5582955061, 7350762742, 2163186255, 5582955075]
hijo2: [1788442041, 2161030013, 1773309963, 2161030013]
Padre1: (2163186255, 5582955075, 1788442041, 2161030013)
Padre2: (1788442041, 5582955082, 4224196989, 5582955061)
hijo1: [2161030013, 1788442041, 5582955082, 4224196989]
hijo2: [5582955061, 2163186255, 5582955075, 1788442041]
Padre1: (2161030013, 7350762713, 4221333375, 4221333605)
Padre2: (1557019791, 1557019715, 2161030013, 7350762713)
hijo1: (2161030013, 7350762713, 4221333375, 4221333605)
hijo2: (1557019791, 1557019715, 2161030013, 7350762713)
Generación: 0
Mejor Individuo: (7350762713, 7350762741, 1773309963, 5582955061)
Hits en caché In

### Algoritmos compactados

In [ ]:
cache = {}

def evaluar(individuo, problema):
    sumPop = 0
    aux = 0
    estrella = AE(problema, Heuristica(problema.velocidad_maxima))

    for candidato in problema.candidatos:
        minimum = 3600*5
        for solucion in individuo:
            tupla = (candidato[0], solucion)
            if tupla in cache:
                val = cache[tupla]
            else:
                estrella.problema.estado_inicial = problema.estados[candidato[0]]
                estrella.problema.estado_final = problema.estados[solucion]
                estrella.start()
                val = estrella.solucion
                cache[tupla] = val
            minimum = min(val, minimum)
        sumPop += candidato[1]
        aux += candidato[1]*minimum
    return (1/sumPop) * aux 

class AlgoritmoAleatorioComprimido():

    cache = {}

    def __init__(self, problema, tamano_poblacion=100):
        self.problema = problema
        self.tamano_poblacion = tamano_poblacion
    
    def algoritmo(self):
        
        mejor_individuo = None
        mejor_fitness = 3600*5

        ids = [x[0] for x in self.problema.candidatos]

        for i in range(self.tamano_poblacion):
            individuo = frozenset(random.sample(ids, self.problema.numero_estaciones))

            if individuo in AlgoritmoAleatorioComprimido.cache:
                fitness = AlgoritmoAleatorioComprimido.cache[individuo]
            else:
                fitness = evaluar(individuo, self.problema)
                AlgoritmoAleatorioComprimido.cache[individuo] = fitness
            
            if mejor_individuo is None:
                mejor_individuo = individuo
                mejor_fitness = fitness
            
            if mejor_fitness > fitness:
                mejor_individuo = individuo
                mejor_fitness = fitness
        
        return mejor_individuo
    
prob = Problema("sample-problems-lab2/medium/calle_palmas_de_gran_canaria_albacete_500_2_candidates_167_ns_23.json")
AAC = AlgoritmoAleatorioComprimido(prob, 1000)
print(AAC.algoritmo())